In [ ]:
"""
Loading the AG_NEWS dataset in bi-gram features format.
"""
!pip install torchtext==0.4
import torch
import torchtext
from torchtext.datasets import text_classification
import os

NGRAMS = 2

if not os.path.isdir('./.data'):
    os.mkdir('./.data')

train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)

BATCH_SIZE = 16

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     |████████████████████████████████| 61kB 10.2MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


ag_news_csv.tar.gz: 11.8MB [00:00, 211MB/s]
120000lines [00:07, 16413.10lines/s]
120000lines [00:14, 8054.20lines/s]
7600lines [00:01, 7454.78lines/s]


In [ ]:
# Import the necessary libraries
import torch.nn as nn
import torch.nn.functional as F

# Creating a class TextClassifier.
class TextClassifier(nn.Module):
    # Defining the __init__() method with proper parameters
    # (vocabulary size, dimensions of the embeddings, number of classes)
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()

        # Defining the embedding layer
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        # Defining the linear forward layer
        self.fc = nn.Linear(embed_dim, num_class)
        # Initializing weights
        self.init_weights()
    # Defining a method to initialize weights.
        # The weights should be random in the range of -0.5 to 0.5.
        # Initializing bias values as zero.
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    # Defining the forward function.
        # This should calculate the embeddings and return the linear layer
        # with calculated embedding values.
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

class TextClassifier2(nn.Module):
	def __init__(self, vocab_size, embed_dim,hidden_dim, num_class):
		super().__init__()


		# self.batch_size = batch_size
		self.num_class = num_class
		self.hidden_size = hidden_dim
		self.vocab_size = vocab_size
		self.embed_dim = embed_dim
		self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)# Initializing the look-up table.
		# self.embedding.weight = nn.Parameter(weights, requires_grad=False) # Assigning the look-up table to the pre-trained GloVe word embedding.
		self.lstm = nn.LSTM(embed_dim, hidden_dim)
		self.label = nn.Linear(hidden_dim, num_class)
		self.init_weights()


    # Defining a method to initialize weights.
        # The weights should be random in the range of -0.5 to 0.5.
        # You can initialize bias values as zero.
	def init_weights(self):

		initrange = 0.5
		self.embedding.weight.data.uniform_(-initrange, initrange)
		self.label.weight.data.uniform_(-initrange, initrange)
		self.label.bias.data.zero_()

	def forward(self, text, offsets):

		embedded = self.embedding(text, offsets)
		lstm_output, (hidden, cell) = self.lstm(embedded.view(len(embedded),1,-1))
		final_output = self.label(lstm_output.view(len(lstm_output),-1))

		return final_output

In [ ]:
'''
Parameters and model instance creation.
'''

# Instantiating the Vocabulary size and the number of classes
# from the training dataset.

VOCAB_SIZE = 1308844
EMBED_DIM = 32
NUM_CLASS = 4
HIDDEN_DIM = 32
# Instantiating the model with the parameters defined above
# Allocating the 'device' variable
model = TextClassifier(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)
model2 = TextClassifier2(VOCAB_SIZE, EMBED_DIM,HIDDEN_DIM, NUM_CLASS).to(device)

In [ ]:
def generate_batch(batch):

    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)

    return text, offsets, label

In [ ]:

def train(train_data):

    # Initial values of training loss and training accuracy

    train_loss = 0
    total_acc = 0

    # Using the PyTorch DataLoader class to load the data
    # into shuffled batches of appropriate sizes into the variable 'data'
    data = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)


    for i, (text, offsets, cls) in enumerate(data):

        # Performing backprop on the optimizer
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

        # Storing the output of the model in variable 'output'
        output= model(text,offsets)
       # output2 = model2(text,offsets)

        # Defining the 'loss' variable (with respect to 'output' and 'cls').
        # Also calculate the total loss in variable 'train_loss'
        loss=criterion(output,cls)
        train_loss += loss.item()

        #loss2=criterion(output2,cls)
        #train_loss2 += loss2.item()



        # Performing the backward propagation on 'loss' and
        # optimizing it through the 'optimizer' step
        loss.backward()


        #loss2.backward()
        optimizer.step()

        # Calculating and storing the total training accuracy
        # in the variable 'total_acc'.
        total_acc += (output.argmax(1) == cls).sum().item()
        #total_acc2 += (output2.argmax(1) == cls).sum().item()


    # Adjusting the learning rate here using the scheduler step

    scheduler.step()

    return train_loss/ len(train_data), total_acc/ len(train_data), output

In [ ]:
def train2(train_data):

    # Initial values of training loss and training accuracy

    train_loss2 = 0
    total_acc2 = 0

    # Using the PyTorch DataLoader class to load the data
    # into shuffled batches of appropriate sizes into the variable 'data'.
    data = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)


    for i, (text, offsets, cls) in enumerate(data):

        # Performing backprop on the optimizer
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

       # Storing the output of the model in variable 'output'
       # output= model(text,offsets)
        output2 = model2(text,offsets)

        # Defining the 'loss' variable (with respect to 'output' and 'cls').
        # Also calculating the total loss in variable 'train_loss'
        loss2=criterion(output2,cls)
        train_loss2 += loss2.item()

        #loss2=criterion(output2,cls)
        #train_loss2 += loss2.item()



        # Performing the backward propagation on 'loss' and
        # optimizing it through the 'optimizer' step
        loss2.backward()


        #loss2.backward()
        optimizer.step()

        # Calculating and storing the total training accuracy
        # in the variable 'total_acc'.
        total_acc2 += (output2.argmax(1) == cls).sum().item()
        #total_acc2 += (output2.argmax(1) == cls).sum().item()


    # Adjust the learning rate here using the scheduler step

    scheduler.step()

    return train_loss2/ len(train_data), total_acc2/ len(train_data), output2

In [ ]:
def test(test_data):

    # Initial values of test loss and test accuracy

    loss = 0
    acc = 0

    # Using DataLoader class to load the data
    # into non-shuffled batches of appropriate sizes.
    data=DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False,
                      collate_fn=generate_batch)


    for text, offsets, cls in data:

        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

        with torch.no_grad():


            # Getting the model output
            output= model(text,offsets)
            #output2= model2(text,offsets)

            #output=(output+output2)/2
            # Calculating and adding the loss to find total 'loss'
            loss_t = criterion(output, cls)
            loss_t += loss_t.item()

            # Calculating the accuracy and storing it in the 'acc' variable
            acc += (output.argmax(1) == cls).sum().item()

    # return loss_t, acc/len(test_data)
    return loss_t/len(test_data), acc/len(test_data), output

def test2(test_data):

    # Initial values of test loss and test accuracy

    loss2 = 0
    acc2 = 0

    # Using DataLoader class to load the data
    # into non-shuffled batches of appropriate sizes.
    data=DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False,
                      collate_fn=generate_batch)


    for text, offsets, cls in data:

        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

        with torch.no_grad():


            # Getting the model output
            output2= model2(text,offsets)
            #output2= model2(text,offsets)

            #output=(output+output2)/2
            # Calculating and add the loss to find total 'loss'
            loss_t2 = criterion(output2, cls)
            loss_t2 += loss_t2.item()

            # Calculating the accuracy and store it in the 'acc' variable
            acc2 += (output2.argmax(1) == cls).sum().item()

    # return loss_t, acc/len(test_data)
    return loss_t2/len(test_data), acc2/len(test_data), output2

In [ ]:
import time
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader


# Setting the number of epochs and the learning rate to
# their initial values here

N_EPOCHS = 10
LEARNING_RATE = 2.0
TRAIN_RATIO = 0.9

# Setting the intial validation loss to positive infinity
ini_valid_loss = float('inf')


# Using the crossEntropy loss function
criterion = torch.nn.CrossEntropyLoss().to(device)


# Using the SGD optimization algorithm with parameters
optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE)


# Using a scheduler function
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, TRAIN_RATIO)


# Splitting the data into train and validation sets using random_split()
train_len = int(len(train_dataset) * 0.8)
train_data, valid_data = random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc , a= train(train_data)
    valid_loss, valid_acc, d = test(valid_data)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')


Epoch: 1  | time in 0 minutes, 8 seconds
	Loss: 0.0323(train)	|	Acc: 80.4%(train)
	Loss: 0.0000(valid)	|	Acc: 89.5%(valid)
Epoch: 2  | time in 0 minutes, 7 seconds
	Loss: 0.0146(train)	|	Acc: 92.2%(train)
	Loss: 0.0000(valid)	|	Acc: 90.7%(valid)
Epoch: 3  | time in 0 minutes, 7 seconds
	Loss: 0.0097(train)	|	Acc: 95.1%(train)
	Loss: 0.0000(valid)	|	Acc: 90.4%(valid)
Epoch: 4  | time in 0 minutes, 7 seconds
	Loss: 0.0064(train)	|	Acc: 97.0%(train)
	Loss: 0.0000(valid)	|	Acc: 90.7%(valid)
Epoch: 5  | time in 0 minutes, 7 seconds
	Loss: 0.0042(train)	|	Acc: 98.2%(train)
	Loss: 0.0000(valid)	|	Acc: 90.9%(valid)
Epoch: 6  | time in 0 minutes, 7 seconds
	Loss: 0.0028(train)	|	Acc: 99.0%(train)
	Loss: 0.0000(valid)	|	Acc: 90.7%(valid)
Epoch: 7  | time in 0 minutes, 7 seconds
	Loss: 0.0020(train)	|	Acc: 99.4%(train)
	Loss: 0.0000(valid)	|	Acc: 90.9%(valid)
Epoch: 8  | time in 0 minutes, 7 seconds
	Loss: 0.0015(train)	|	Acc: 99.6%(train)
	Loss: 0.0000(valid)	|	Acc: 91.0%(valid)
Epoch: 9  | time

In [ ]:
import time
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader


# Setting the number of epochs and the learning rate to
# their initial values here

N_EPOCHS = 10
LEARNING_RATE = 2.0
TRAIN_RATIO = 0.9


# Setting the intial validation loss to positive infinity
ini_valid_loss = float('inf')


# Using the CrossEntropy loss function
criterion = torch.nn.CrossEntropyLoss().to(device)


# Using the SGD optimization algorithm with parameters
optimizer = torch.optim.SGD(model2.parameters(), LEARNING_RATE)


# Using a scheduler function
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, TRAIN_RATIO)


# Splitting the data into train and validation sets using random_split()
train_len = int(len(train_dataset) * 0.9)
train_data, valid_data = random_split(train_dataset, [train_len, len(train_dataset) - train_len])


for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc, a1 = train2(train_data)
    valid_loss, valid_acc, d1 = test2(valid_data)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 16 seconds
	Loss: 0.0624(train)	|	Acc: 52.1%(train)
	Loss: 0.0001(valid)	|	Acc: 84.9%(valid)
Epoch: 2  | time in 0 minutes, 16 seconds
	Loss: 0.0199(train)	|	Acc: 89.0%(train)
	Loss: 0.0000(valid)	|	Acc: 89.7%(valid)
Epoch: 3  | time in 0 minutes, 16 seconds
	Loss: 0.0128(train)	|	Acc: 93.2%(train)
	Loss: 0.0000(valid)	|	Acc: 90.8%(valid)
Epoch: 4  | time in 0 minutes, 16 seconds
	Loss: 0.0090(train)	|	Acc: 95.3%(train)
	Loss: 0.0000(valid)	|	Acc: 90.7%(valid)
Epoch: 5  | time in 0 minutes, 16 seconds
	Loss: 0.0063(train)	|	Acc: 96.8%(train)
	Loss: 0.0000(valid)	|	Acc: 89.4%(valid)
Epoch: 6  | time in 0 minutes, 16 seconds
	Loss: 0.0045(train)	|	Acc: 97.8%(train)
	Loss: 0.0000(valid)	|	Acc: 90.6%(valid)
Epoch: 7  | time in 0 minutes, 16 seconds
	Loss: 0.0031(train)	|	Acc: 98.5%(train)
	Loss: 0.0001(valid)	|	Acc: 90.1%(valid)
Epoch: 8  | time in 0 minutes, 16 seconds
	Loss: 0.0023(train)	|	Acc: 98.9%(train)
	Loss: 0.0001(valid)	|	Acc: 90.4%(valid)
Epoch: 9

In [ ]:
def comb(test_data):

    # Initial values of test loss and test accuracy

    loss = 0
    acc = 0

    # Using DataLoader class to load the data
    # into non-shuffled batches of appropriate sizes
    data=DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False,
                      collate_fn=generate_batch)


    for text, offsets, cls in data:

        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

        with torch.no_grad():


            # Getting the model outputs form simple fully connected network and LSTM network.
            output= model(text,offsets)
            output2= model2(text,offsets)

           # Combining the two predictions using dempster shafer combination rule please see the report
            a= output
            b= output2

            a=a.cpu()
            b=b.cpu()

            #mass of model a
            x= F.softmax(a, dim=1)
            #mass of model b
            y= F.softmax(b, dim=1)



            x= x.numpy()
            y= y.numpy()
            x = x.astype(float)
            y= y.astype(float)
            #print(np.sum(x[,1:4]))

            # we are considering 60% weightage for the model a , Fully connected network
            tempy= np.delete(x, 0, 1)
            tx1a = (0.6*(x[:,0]))
            fx1a = 0.6*(np.sum(tempy, axis=1))
            ux1a = (0.4*(np.sum(tempy, axis=1)))+ 0.4 * (x[:,0])


            tempy= np.delete(x, 1, 1)
            tx2a = (0.6*(x[:,1]))
            fx2a = 0.6*(np.sum(tempy, axis=1))
            ux2a = (0.4*(np.sum(tempy, axis=1)))+ 0.4 * (x[:,1])

            tempy= np.delete(x, 2, 1)
            tx3a = (0.6*(x[:,2]))
            fx3a = 0.6*(np.sum(tempy, axis=1))
            ux3a = (0.4*(np.sum(tempy, axis=1)))+ 0.4 * (x[:,2])


            tempy= np.delete(x, 3, 1)
            tx4a = (0.6*(x[:,3]))
            fx4a = 0.6*(np.sum(tempy, axis=1))
            ux4a = (0.4*(np.sum(tempy, axis=1)))+ 0.4 * (x[:,3])

# we are considering 80% weightage for the model b , LSTM network
            #calculating true, uncertainity and false probability for class 1
            tempy= np.delete(y, 0, 1)
            tx1b = (0.8*(y[:,0]))
            fx1b = 0.8*(np.sum(tempy, axis=1))
            ux1b = (0.2*(np.sum(tempy, axis=1)))+ 0.2 * (y[:,0])

          #calculating true, uncertainity and false probability for class 2
            tempy= np.delete(y, 1, 1)
            tx2b = (0.8*(y[:,1]))
            fx2b = 0.8*(np.sum(tempy, axis=1))
            ux2b = (0.2*(np.sum(tempy, axis=1)))+ 0.2 * (y[:,1])

            #calculating true, uncertainity and false probability for class 3
            tempy= np.delete(y, 2, 1)
            tx3b = (0.8*(y[:,2]))
            fx3b = 0.8*(np.sum(tempy, axis=1))
            ux3b = (0.2*(np.sum(tempy, axis=1)))+ 0.2 * (y[:,2])

            ##calculating true, uncertainity and false probability for class 4
            tempy= np.delete(y, 3, 1)
            tx4b = (0.8*(y[:,3]))
            fx4b = 0.8*(np.sum(tempy, axis=1))
            ux4b = (0.2*(np.sum(tempy, axis=1)))+ 0.2 * (y[:,3])

# combining the probabilities of truth using dempster shafer combination rule for all the classes using clculated true , uncertainity and flase
#probabilities

            comb1 = (tx1a * tx1b ) + (tx1a * ux1b) + (ux1a * tx1a )
            mmm =   (ux1a * fx1b)  + (fx1a * ux1b) + (fx1a * fx1b )
            comb1= comb1/ (1 - mmm)

            comb2 = (tx2a * tx2b ) + (tx2a * ux2b) + (ux2a * tx2a )
            mmm =   (ux2a * fx2b)  + (fx2a * ux2b) + (fx2a * fx2b )
            comb2= comb2/ (1 - mmm)

            comb3 = (tx3a * tx3b ) + (tx3a * ux3b) + (ux3a * tx3a )
            mmm =   (ux3a * fx3b)  + (fx3a * ux3b) + (fx3a * fx3b )
            comb3= comb3/ (1 - mmm)

            comb4 = (tx4a * tx4b ) + (tx4a * ux4b) + (ux4a * tx4a )
            mmm =   (ux4a * fx4b)  + (fx4a * ux4b) + (fx4a * fx4b )
            comb4= comb4/ (1 - mmm)

            #combined probability for testing
            final= np.matrix([comb1, comb2, comb3, comb4])
            final= final.T
            final= final.astype(float)
            final= torch.tensor(final)

            #converting the range to make differences bigger among combined probabilities
            OldRange = (final.max() - final.min())
            NewRange = (50.0 - 1.0)
            NewValue = (((final - final.min()) * NewRange) / OldRange) + 1.0
            output3= NewValue.to(device)


            # Calculating and adding the loss to find total 'loss'
            loss_t = criterion(output3, cls)
            loss_t += loss_t.item()

            # Calculating the accuracy and store it in the 'acc' variable
            acc += (output3.argmax(1) == cls).sum().item()

    # returning loss_t, acc/len(test_data)
    return loss_t/len(test_data), acc/len(test_data)



In [ ]:
# the results (loss and accuracy) on the test data
import numpy as np
print('Checking the results of test dataset...')
test_loss, test_acc, a= test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc fully connected model: {test_acc * 100:.1f}%(test)')
test_loss, test_acc, b = test2(test_dataset)

print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc LSTM model: {test_acc * 100:.1f}%(test)')

test_loss, test_acc = comb(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc combined prediction using DSCR: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0003(test)	|	Acc fully connected model: 89.1%(test)
	Loss: 0.0003(test)	|	Acc LSTM model: 89.7%(test)
	Loss: 0.0014(test)	|	Acc combined prediction using DSCR: 89.1%(test)
